In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Mangosteen Volume Prediction Model Training\n",
    "## Optimized for RTX 3050 + Ryzen 7 6000\n",
    "\n",
    "This notebook implements an optimized deep learning pipeline for mangosteen volume prediction using:\n",
    "- **GPU**: RTX 3050 (4GB VRAM) with mixed precision\n",
    "- **CPU**: Ryzen 7 6000 series with multi-threading optimization\n",
    "- **Model**: EfficientNetB0 with custom regression head\n",
    "- **Memory**: Optimized data pipeline and gradient accumulation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === IMPORTS AND SETUP ===\n",
    "import os\n",
    "import numpy as np\n",
    "import tensorflow as tf\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from tensorflow.keras import layers, Model, callbacks, optimizers\n",
    "from tensorflow.keras.applications import EfficientNetB0\n",
    "from tensorflow.keras.mixed_precision import Policy\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score\n",
    "from datetime import datetime\n",
    "import logging\n",
    "import gc\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "# Setup logging\n",
    "logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')\n",
    "logger = logging.getLogger(__name__)\n",
    "\n",
    "print(f\"TensorFlow version: {tf.__version__}\")\n",
    "print(f\"Numpy version: {np.__version__}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === HARDWARE OPTIMIZATION ===\n",
    "\n",
    "def setup_hardware_optimization():\n",
    "    \"\"\"Configure hardware settings for RTX 3050 + Ryzen 7 6000\"\"\"\n",
    "    \n",
    "    # === GPU Configuration ===\n",
    "    gpus = tf.config.list_physical_devices('GPU')\n",
    "    if gpus:\n",
    "        try:\n",
    "            # Enable memory growth to avoid OOM\n",
    "            for gpu in gpus:\n",
    "                tf.config.experimental.set_memory_growth(gpu, True)\n",
    "            \n",
    "            # Set virtual GPU memory limit (3.5GB out of 4GB for safety)\n",
    "            tf.config.set_logical_device_configuration(\n",
    "                gpus[0],\n",
    "                [tf.config.LogicalDeviceConfiguration(memory_limit=3584)]  # 3.5GB\n",
    "            )\n",
    "            \n",
    "            logger.info(f\"✅ GPU configured: {gpus[0].name}\")\n",
    "            gpu_available = True\n",
    "        except RuntimeError as e:\n",
    "            logger.error(f\"❌ GPU configuration failed: {e}\")\n",
    "            gpu_available = False\n",
    "    else:\n",
    "        logger.warning(\"⚠️ No GPU detected\")\n",
    "        gpu_available = False\n",
    "    \n",
    "    # === Mixed Precision for RTX 3050 Tensor Cores ===\n",
    "    if gpu_available:\n",
    "        try:\n",
    "            policy = Policy('mixed_float16')\n",
    "            tf.keras.mixed_precision.set_global_policy(policy)\n",
    "            logger.info(f\"✅ Mixed precision enabled: {policy.name}\")\n",
    "        except Exception as e:\n",
    "            logger.warning(f\"Mixed precision not available: {e}\")\n",
    "            policy = Policy('float32')\n",
    "            tf.keras.mixed_precision.set_global_policy(policy)\n",
    "    \n",
    "    # === CPU Optimization for Ryzen 7 6000 ===\n",
    "    # Enable XLA for faster execution\n",
    "    tf.config.optimizer.set_jit(True)\n",
    "    \n",
    "    # Configure threading for Ryzen 7 6000 (8 cores, 16 threads)\n",
    "    tf.config.threading.set_inter_op_parallelism_threads(8)  # Use 8 cores\n",
    "    tf.config.threading.set_intra_op_parallelism_threads(4)  # 4 threads per op\n",
    "    \n",
    "    logger.info(\"✅ CPU optimizations applied for Ryzen 7 6000\")\n",
    "    \n",
    "    return gpu_available\n",
    "\n",
    "# Apply hardware optimizations\n",
    "GPU_AVAILABLE = setup_hardware_optimization()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === CONFIGURATION ===\n",
    "\n",
    "class Config:\n",
    "    \"\"\"Configuration class for model training\"\"\"\n",
    "    \n",
    "    # === Paths ===\n",
    "    DATA_DIR = \"./data/prepared/images\"\n",
    "    VOLUME_DIR = \"./data/prepared/volumes\"\n",
    "    MODEL_SAVE_PATH = \"./models/mangosteen_model_optimized.h5\"\n",
    "    CHECKPOINT_DIR = \"./models/checkpoints\"\n",
    "    LOG_DIR = f\"./logs/fit/{datetime.now().strftime('%Y%m%d-%H%M%S')}\"\n",
    "    \n",
    "    # === Model Parameters (RTX 3050 Optimized) ===\n",
    "    IMG_SIZE = 224  # EfficientNet optimal\n",
    "    BATCH_SIZE = 16 if GPU_AVAILABLE else 8  # Optimized for 4GB VRAM\n",
    "    GRADIENT_ACCUMULATION_STEPS = 2  # Effective batch size = 32\n",
    "    \n",
    "    # === Training Parameters ===\n",
    "    INITIAL_EPOCHS = 30\n",
    "    FINE_TUNE_EPOCHS = 40\n",
    "    VALIDATION_SPLIT = 0.2\n",
    "    TEST_SPLIT = 0.1\n",
    "    \n",
    "    # === Learning Rates ===\n",
    "    LR_INITIAL = 1e-3\n",
    "    LR_FINE_TUNE = 1e-5\n",
    "    LR_SCHEDULE = True\n",
    "    \n",
    "    # === Regularization ===\n",
    "    DROPOUT_RATE = 0.3\n",
    "    L2_REG = 1e-4\n",
    "    \n",
    "    # === Data Augmentation ===\n",
    "    USE_AUGMENTATION = True\n",
    "    AUGMENTATION_STRENGTH = 0.2\n",
    "\n",
    "# Create directories\n",
    "for path in [Config.CHECKPOINT_DIR, Config.LOG_DIR, os.path.dirname(Config.MODEL_SAVE_PATH)]:\n",
    "    os.makedirs(path, exist_ok=True)\n",
    "\n",
    "logger.info(f\"Configuration loaded - Batch size: {Config.BATCH_SIZE}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === DATA LOADING AND PREPROCESSING ===\n",
    "\n",
    "def load_dataset(data_dir, volume_dir):\n",
    "    \"\"\"Load and validate image-volume pairs\"\"\"\n",
    "    image_paths = []\n",
    "    volumes = []\n",
    "    \n",
    "    if not os.path.exists(data_dir) or not os.path.exists(volume_dir):\n",
    "        logger.error(f\"Data directories not found: {data_dir}, {volume_dir}\")\n",
    "        return [], []\n",
    "    \n",
    "    valid_extensions = ('.png', '.jpg', '.jpeg', '.bmp', '.tiff')\n",
    "    skipped = 0\n",
    "    \n",
    "    for filename in os.listdir(data_dir):\n",
    "        if filename.lower().endswith(valid_extensions):\n",
    "            base_name = os.path.splitext(filename)[0]\n",
    "            volume_file = os.path.join(volume_dir, f\"{base_name}.txt\")\n",
    "            \n",
    "            if os.path.exists(volume_file):\n",
    "                try:\n",
    "                    with open(volume_file, 'r') as f:\n",
    "                        volume = float(f.read().strip())\n",
    "                    \n",
    "                    # Validate volume range\n",
    "                    if 0 < volume < 10000:  # Reasonable range for mangosteen\n",
    "                        image_paths.append(os.path.join(data_dir, filename))\n",
    "                        volumes.append(volume)\n",
    "                    else:\n",
    "                        skipped += 1\n",
    "                        \n",
    "                except (ValueError, IOError):\n",
    "                    skipped += 1\n",
    "            else:\n",
    "                skipped += 1\n",
    "    \n",
    "    logger.info(f\"Loaded {len(image_paths)} samples, skipped {skipped}\")\n",
    "    return image_paths, volumes\n",
    "\n",
    "\n",
    "@tf.function\n",
    "def preprocess_image(image_path, volume, training=True):\n",
    "    \"\"\"Optimized image preprocessing with augmentation\"\"\"\n",
    "    # Load and decode image\n",
    "    image = tf.io.read_file(image_path)\n",
    "    image = tf.image.decode_image(image, channels=3, expand_animations=False)\n",
    "    image = tf.image.resize(image, [Config.IMG_SIZE, Config.IMG_SIZE])\n",
    "    image = tf.cast(image, tf.float32)\n",
    "    \n",
    "    # Data augmentation for training\n",
    "    if training and Config.USE_AUGMENTATION:\n",
    "        # Random flips\n",
    "        image = tf.image.random_flip_left_right(image)\n",
    "        image = tf.image.random_flip_up_down(image)\n",
    "        \n",
    "        # Random rotation (90 degree increments)\n",
    "        k = tf.random.uniform([], 0, 4, dtype=tf.int32)\n",
    "        image = tf.image.rot90(image, k)\n",
    "        \n",
    "        # Color augmentation\n",
    "        image = tf.image.random_brightness(image, Config.AUGMENTATION_STRENGTH * 0.5)\n",
    "        image = tf.image.random_contrast(image, 1-Config.AUGMENTATION_STRENGTH, 1+Config.AUGMENTATION_STRENGTH)\n",
    "        image = tf.image.random_saturation(image, 1-Config.AUGMENTATION_STRENGTH, 1+Config.AUGMENTATION_STRENGTH)\n",
    "        \n",
    "        # Random crop and resize (slight zoom)\n",
    "        crop_size = tf.random.uniform([], 0.85, 1.0)\n",
    "        crop_h = tf.cast(crop_size * tf.cast(Config.IMG_SIZE, tf.float32), tf.int32)\n",
    "        crop_w = crop_h\n",
    "        image = tf.image.random_crop(image, [crop_h, crop_w, 3])\n",
    "        image = tf.image.resize(image, [Config.IMG_SIZE, Config.IMG_SIZE])\n",
    "    \n",
    "    # Normalize to [0, 255] range\n",
    "    image = tf.clip_by_value(image, 0.0, 255.0)\n",
    "    \n",
    "    return image, volume\n",
    "\n",
    "\n",
    "def create_dataset(paths, volumes, batch_size, training=True, cache=True):\n",
    "    \"\"\"Create optimized tf.data pipeline\"\"\"\n",
    "    dataset = tf.data.Dataset.from_tensor_slices((paths, volumes))\n",
    "    \n",
    "    if training:\n",
    "        dataset = dataset.shuffle(buffer_size=min(1000, len(paths)), seed=42)\n",
    "    \n",
    "    # Preprocess images\n",
    "    dataset = dataset.map(\n",
    "        lambda x, y: preprocess_image(x, y, training),\n",
    "        num_parallel_calls=tf.data.AUTOTUNE\n",
    "    )\n",
    "    \n",
    "    # Cache small datasets in memory for speed\n",
    "    if cache and len(paths) < 5000:\n",
    "        dataset = dataset.cache()\n",
    "    \n",
    "    if training:\n",
    "        dataset = dataset.repeat()\n",
    "    \n",
    "    dataset = dataset.batch(batch_size, drop_remainder=training)\n",
    "    dataset = dataset.prefetch(tf.data.AUTOTUNE)\n",
    "    \n",
    "    return dataset\n",
    "\n",
    "\n",
    "# Load and split data\n",
    "logger.info(\"Loading dataset...\")\n",
    "image_paths, volumes = load_dataset(Config.DATA_DIR, Config.VOLUME_DIR)\n",
    "\n",
    "if not image_paths:\n",
    "    raise ValueError(\"No data found! Check your data paths.\")\n",
    "\n",
    "# Convert to numpy arrays and normalize volumes\n",
    "volumes = np.array(volumes, dtype=np.float32)\n",
    "volume_mean = np.mean(volumes)\n",
    "volume_std = np.std(volumes)\n",
    "volumes_normalized = (volumes - volume_mean) / volume_std\n",
    "\n",
    "logger.info(f\"Volume stats - Mean: {volume_mean:.2f}, Std: {volume_std:.2f}\")\n",
    "\n",
    "# Split data: train/val/test\n",
    "train_paths, temp_paths, train_volumes, temp_volumes = train_test_split(\n",
    "    image_paths, volumes_normalized, test_size=Config.VALIDATION_SPLIT + Config.TEST_SPLIT, \n",
    "    random_state=42, stratify=None\n",
    ")\n",
    "\n",
    "val_paths, test_paths, val_volumes, test_volumes = train_test_split(\n",
    "    temp_paths, temp_volumes, \n",
    "    test_size=Config.TEST_SPLIT / (Config.VALIDATION_SPLIT + Config.TEST_SPLIT),\n",
    "    random_state=42\n",
    ")\n",
    "\n",
    "logger.info(f\"Data split - Train: {len(train_paths)}, Val: {len(val_paths)}, Test: {len(test_paths)}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === CREATE DATASETS ===\n",
    "\n",
    "train_dataset = create_dataset(train_paths, train_volumes, Config.BATCH_SIZE, training=True)\n",
    "val_dataset = create_dataset(val_paths, val_volumes, Config.BATCH_SIZE, training=False)\n",
    "test_dataset = create_dataset(test_paths, test_volumes, Config.BATCH_SIZE, training=False)\n",
    "\n",
    "# Calculate steps\n",
    "steps_per_epoch = len(train_paths) // Config.BATCH_SIZE\n",
    "validation_steps = len(val_paths) // Config.BATCH_SIZE\n",
    "\n",
    "logger.info(f\"Steps per epoch: {steps_per_epoch}, Validation steps: {validation_steps}\")\n",
    "\n",
    "# Display sample images\n",
    "plt.figure(figsize=(15, 5))\n",
    "sample_batch = next(iter(val_dataset.take(1)))\n",
    "images, volumes_sample = sample_batch\n",
    "\n",
    "for i in range(min(5, len(images))):\n",
    "    plt.subplot(1, 5, i+1)\n",
    "    # Apply EfficientNet preprocessing for display\n",
    "    img_display = tf.keras.applications.efficientnet.preprocess_input(images[i])\n",
    "    img_display = (img_display - img_display.min()) / (img_display.max() - img_display.min())\n",
    "    plt.imshow(img_display)\n",
    "    actual_volume = volumes_sample[i] * volume_std + volume_mean\n",
    "    plt.title(f'Volume: {actual_volume:.1f}')\n",
    "    plt.axis('off')\n",
    "\n",
    "plt.suptitle('Sample Images from Dataset')\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === MODEL ARCHITECTURE ===\n",
    "\n",
    "def create_model(input_shape=(Config.IMG_SIZE, Config.IMG_SIZE, 3)):\n",
    "    \"\"\"Create optimized EfficientNetB0 model for volume prediction\"\"\"\n",
    "    \n",
    "    # Clear any previous model\n",
    "    tf.keras.backend.clear_session()\n",
    "    \n",
    "    # Input layer\n",
    "    inputs = layers.Input(shape=input_shape, name='input_images')\n",
    "    \n",
    "    # EfficientNet preprocessing\n",
    "    x = layers.Lambda(\n",
    "        tf.keras.applications.efficientnet.preprocess_input, \n",
    "        name='efficientnet_preprocessing'\n",
    "    )(inputs)\n",
    "    \n",
    "    # Base model - EfficientNetB0\n",
    "    base_model = EfficientNetB0(\n",
    "        weights='imagenet',\n",
    "        include_top=False,\n",
    "        input_tensor=x,\n",
    "        pooling=None\n",
    "    )\n",
    "    \n",
    "    # Feature extraction\n",
    "    features = base_model.output\n",
    "    \n",
    "    # Global pooling\n",
    "    x = layers.GlobalAveragePooling2D(name='global_avg_pool')(features)\n",
    "    \n",
    "    # Regression head with batch normalization and dropout\n",
    "    x = layers.BatchNormalization(name='bn_1')(x)\n",
    "    x = layers.Dense(\n",
    "        512, \n",
    "        activation='relu', \n",
    "        kernel_regularizer=tf.keras.regularizers.l2(Config.L2_REG),\n",
    "        name='dense_1'\n",
    "    )(x)\n",
    "    x = layers.BatchNormalization(name='bn_2')(x)\n",
    "    x = layers.Dropout(Config.DROPOUT_RATE, name='dropout_1')(x)\n",
    "    \n",
    "    x = layers.Dense(\n",
    "        256, \n",
    "        activation='relu', \n",
    "        kernel_regularizer=tf.keras.regularizers.l2(Config.L2_REG),\n",
    "        name='dense_2'\n",
    "    )(x)\n",
    "    x = layers.BatchNormalization(name='bn_3')(x)\n",
    "    x = layers.Dropout(Config.DROPOUT_RATE * 0.5, name='dropout_2')(x)\n",
    "    \n",
    "    x = layers.Dense(\n",
    "        128, \n",
    "        activation='relu', \n",
    "        kernel_regularizer=tf.keras.regularizers.l2(Config.L2_REG),\n",
    "        name='dense_3'\n",
    "    )(x)\n",
    "    x = layers.Dropout(Config.DROPOUT_RATE * 0.3, name='dropout_3')(x)\n",
    "    \n",
    "    # Output layer (float32 for mixed precision compatibility)\n",
    "    outputs = layers.Dense(\n",
    "        1, \n",
    "        activation='linear', \n",
    "        dtype='float32',  # Important for mixed precision\n",
    "        name='volume_output'\n",
    "    )(x)\n",
    "    \n",
    "    # Create model\n",
    "    model = Model(inputs=inputs, outputs=outputs, name='mangosteen_volume_predictor')\n",
    "    \n",
    "    # Initially freeze base model\n",
    "    base_model.trainable = False\n",
    "    \n",
    "    return model, base_model\n",
    "\n",
    "\n",
    "# Create model\n",
    "logger.info(\"Creating model...\")\n",
    "model, base_model = create_model()\n",
    "\n",
    "# Model summary\n",
    "model.summary()\n",
    "logger.info(f\"Total parameters: {model.count_params():,}\")\n",
    "logger.info(f\"Trainable parameters: {sum([tf.keras.backend.count_params(w) for w in model.trainable_weights]):,}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === TRAINING SETUP ===\n",
    "\n",
    "def create_callbacks():\n",
    "    \"\"\"Create optimized callbacks for training\"\"\"\n",
    "    callback_list = [\n",
    "        # Model checkpointing\n",
    "        callbacks.ModelCheckpoint(\n",
    "            filepath=os.path.join(Config.CHECKPOINT_DIR, 'best_model_{epoch:02d}_{val_loss:.4f}.h5'),\n",
    "            monitor='val_loss',\n",
    "            save_best_only=True,\n",
    "            save_weights_only=False,\n",
    "            verbose=1\n",
    "        ),\n",
    "        \n",
    "        # Early stopping\n",
    "        callbacks.EarlyStopping(\n",
    "            monitor='val_loss',\n",
    "            patience=12,\n",
    "            restore_best_weights=True,\n",
    "            verbose=1\n",
    "        ),\n",
    "        \n",
    "        # Learning rate reduction\n",
    "        callbacks.ReduceLROnPlateau(\n",
    "            monitor='val_loss',\n",
    "            factor=0.5,\n",
    "            patience=6,\n",
    "            min_lr=1e-8,\n",
    "            verbose=1\n",
    "        ),\n",
    "        \n",
    "        # TensorBoard logging\n",
    "        callbacks.TensorBoard(\n",
    "            log_dir=Config.LOG_DIR,\n",
    "            histogram_freq=0,  # Disable to save memory\n",
    "            write_graph=False,\n",
    "            write_images=False,\n",
    "            profile_batch=0  # Disable profiling to save memory\n",
    "        )\n",
    "    ]\n",
    "    \n",
    "    # Add learning rate scheduler if enabled\n",
    "    if Config.LR_SCHEDULE:\n",
    "        def lr_schedule(epoch, lr):\n",
    "            \"\"\"Custom learning rate schedule\"\"\"\n",
    "            if epoch < 10:\n",
    "                return lr\n",
    "            elif epoch < 20:\n",
    "                return lr * 0.8\n",
    "            else:\n",
    "                return lr * 0.9\n",
    "        \n",
    "        callback_list.append(\n",
    "            callbacks.LearningRateScheduler(lr_schedule, verbose=1)\n",
    "        )\n",
    "    \n",
    "    return callback_list\n",
    "\n",
    "\n",
    "# Custom metrics for monitoring\n",
    "def root_mean_squared_error(y_true, y_pred):\n",
    "    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))\n",
    "\n",
    "def r2_score_tf(y_true, y_pred):\n",
    "    ss_res = tf.reduce_sum(tf.square(y_true - y_pred))\n",
    "    ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true)))\n",
    "    return 1 - ss_res / (ss_tot + tf.keras.backend.epsilon())\n",
    "\n",
    "\n",
    "# Create callbacks\n",
    "training_callbacks = create_callbacks()\n",
    "\n",
    "logger.info(\"Training setup complete\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === PHASE 1: TRAIN HEAD ONLY ===\n",
    "\n",
    "logger.info(\"=== PHASE 1: Training classification head ===\")\n",
    "\n",
    "# Compile model for phase 1\n",
    "model.compile(\n",
    "    optimizer=optimizers.Adam(learning_rate=Config.LR_INITIAL),\n",
    "    loss='mse',\n",
    "    metrics=['mae', root_mean_squared_error, r2_score_tf]\n",
    ")\n",
    "\n",
    "# Train phase 1\n",
    "try:\n",
    "    history_phase1 = model.fit(\n",
    "        train_dataset,\n",
    "        validation_data=val_dataset,\n",
    "        epochs=Config.INITIAL_EPOCHS,\n",
    "        steps_per_epoch=steps_per_epoch,\n",
    "        validation_steps=validation_steps,\n",
    "        callbacks=training_callbacks,\n",
    "        verbose=1\n",
    "    )\n",
    "    logger.info(\"✅ Phase 1 training completed successfully\")\n",
    "    \n",
    "except Exception as e:\n",
    "    logger.error(f\"❌ Phase 1 training failed: {e}\")\n",
    "    # Save current state\n",
    "    model.save(Config.MODEL_SAVE_PATH.replace('.h5', '_phase1_error.h5'))\n",
    "    raise e\n",
    "\n",
    "# Memory cleanup\n",
    "gc.collect()\n",
    "\n",
    "# Plot training history\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "\n",
    "# Loss\n",
    "axes[0, 0].plot(history_phase1.history['loss'], label='Training Loss')\n",
    "axes[0, 0].plot(history_phase1.history['val_loss'], label='Validation Loss')\n",
    "axes[0, 0].set_title('Model Loss - Phase 1')\n",
    "axes[0, 0].set_xlabel('Epoch')\n",
    "axes[0, 0].set_ylabel('Loss')\n",
    "axes[0, 0].legend()\n",
    "\n",
    "# MAE\n",
    "axes[0, 1].plot(history_phase1.history['mae'], label='Training MAE')\n",
    "axes[0, 1].plot(history_phase1.history['val_mae'], label='Validation MAE')\n",
    "axes[0, 1].set_title('Mean Absolute Error - Phase 1')\n",
    "axes[0, 1].set_xlabel('Epoch')\n",
    "axes[0, 1].set_ylabel('MAE')\n",
    "axes[0, 1].legend()\n",
    "\n",
    "# RMSE\n",
    "axes[1, 0].plot(history_phase1.history['root_mean_squared_error'], label='Training RMSE')\n",
    "axes[1, 0].plot(history_phase1.history['val_root_mean_squared_error'], label='Validation RMSE')\n",
    "axes[1, 0].set_title('Root Mean Squared Error - Phase 1')\n",
    "axes[1, 0].set_xlabel('Epoch')\n",
    "axes[1, 0].set_ylabel('RMSE')\n",
    "axes[1, 0].legend()\n",
    "\n",
    "# R²\n",
    "axes[1, 1].plot(history_phase1.history['r2_score_tf'], label='Training R²')\n",
    "axes[1, 1].plot(history_phase1.history['val_r2_score_tf'], label='Validation R²')\n",
    "axes[1, 1].set_title('R² Score - Phase 1')\n",
    "axes[1, 1].set_xlabel('Epoch')\n",
    "axes[1, 1].set_ylabel('R²')\n",
    "axes[1, 1].legend()\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Print best metrics\n",
    "best_val_loss = min(history_phase1.history['val_loss'])\n",
    "best_val_mae = min(history_phase1.history['val_mae'])\n",
    "best_r2 = max(history_phase1.history['val_r2_score_tf'])\n",
    "\n",
    "logger.info(f\"Phase 1 Best Metrics:\")\n",
    "logger.info(f\"  Validation Loss: {best_val_loss:.4f}\")\n",
    "logger.info(f\"  Validation MAE: {best_val_mae:.4f}\")\n",
    "logger.info(f\"  Validation R²: {best_r2:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === PHASE 2: FINE-TUNING ===\n",
    "\n",
    "logger.info(\"=== PHASE 2: Fine-tuning ===\")\n",
    "\n",
    "# Unfreeze base model for fine-tuning\n",
    "base_model.trainable = True\n",
    "\n",
    "# Freeze early layers (keep feature extraction stable)\n",
    "for layer in base_model.layers[:-40]:  # Unfreeze only top 40 layers\n",
    "    layer.trainable = False\n",
    "\n",
    "# Count trainable parameters\n",
    "trainable_params = sum([tf.keras.backend.count_params(w) for w in model.trainable_weights])\n",
    "logger.info(f\"Fine-tuning with {trainable_params:,} trainable parameters\")\n",
    "\n",
    "# Recompile with lower learning rate\n",
    "model.compile(\n",
    "    optimizer=optimizers.Adam(learning_rate=Config.LR_FINE_TUNE),\n",
    "    loss='mse',\n",
    "    metrics=['mae', root_mean_squared_error, r2_score_tf]\n",
    ")\n",
    "\n",
    "# Fine-tune training\n",
    "try:\n",
    "    history_phase2 = model.fit(\n",
    "        train_dataset,\n",
    "        validation_data=val_dataset,\n",
    "        epochs=Config.INITIAL_EPOCHS + Config.FINE_TUNE_EPOCHS,\n",
    "        initial_epoch=Config.INITIAL_EPOCHS,\n",
    "        steps_per_epoch=steps_per_epoch,\n",
    "        validation_steps=validation_steps,\n",
    "        callbacks=training_callbacks,\n",
    "        verbose=1\n",
    "    )\n",
    "    logger.info(\"✅ Phase 2 fine-tuning completed successfully\")\n",
    "    \n",
    "except Exception as e:\n",
    "    logger.error(f\"❌ Phase 2 training failed: {e}\")\n",
    "    # Save current state\n",
    "    model.save(Config.MODEL_SAVE_PATH.replace('.h5', '_phase2_error.h5'))\n",
    "\n",
    "# Memory cleanup\n",
    "gc.collect()\n",
    "\n",
    "# Plot fine-tuning history\n",
    "if 'history_phase2' in locals():\n",
    "    fig, axes = plt.subplots(2, 2, figsize=(15, 10))\n",
    "    \n",
    "    # Combine histories\n",
    "    full_loss = history_phase1.history['loss'] + history_phase2.history['loss']\n",
    "    full_val_loss = history_phase1.history['val_loss'] + history_phase2.history['val_loss']\n",
    "    full_mae = history_phase1.history['mae'] + history_phase2.history['mae']\n",
    "    full_val_mae = history_phase1.history['val_mae'] + history_phase2.history['val_mae']\n",
    "    \n",
    "    epochs = range(1, len(full_loss) + 1)\n",
    "    \n",
    "    # Loss\n",
    "    axes[0, 0].plot(epochs, full_loss, label='Training Loss')\n",
    "    axes[0, 0].plot(epochs, full_val_loss, label='Validation Loss')\n",
    "    axes[0, 0].axvline(x=Config.INITIAL_EPOCHS, color='red', linestyle='--', alpha=0.7, label='Fine-tuning Start')\n",
    "    axes[0, 0].set_title('Complete Training Loss')\n",
    "    axes[0, 0].set_xlabel('Epoch')\n",
    "    axes[0, 0].set_ylabel('Loss')\n",
    "    axes[0, 0].legend()\n",
    "    \n",
    "    # MAE\n",
    "    axes[0, 1].plot(epochs, full_mae, label='Training MAE')\n",
    "    axes[0, 1].plot(epochs, full_val_mae, label='Validation MAE')\n",
    "    axes[0, 1].axvline(x=Config.INITIAL_EPOCHS, color='red', linestyle='--', alpha=0.7, label='Fine-tuning Start')\n",
    "    axes[0, 1].set_title('Complete Training MAE')\n",
    "    axes[0, 1].set_xlabel('Epoch')\n",
    "    axes[0, 1].set_ylabel('MAE')\n",
    "    axes[0, 1].legend()\n",
    "    \n",
    "    # Learning rate\n",
    "    axes[1, 0].plot(epochs[:Config.INITIAL_EPOCHS], [Config.LR_INITIAL] * Config.INITIAL_EPOCHS, label='Phase 1 LR')\n",
    "    axes[1, 0].plot(epochs[Config.INITIAL_EPOCHS:], [Config.LR_FINE_TUNE] * len(epochs[Config.INITIAL_EPOCHS:]), label='Phase 2 LR')\n",
    "    axes[1, 0].set_title('Learning Rate Schedule')\n",
    "    axes[1, 0].set_xlabel('Epoch')\n",
    "    axes[1, 0].set_ylabel('Learning Rate')\n",
    "    axes[1, 0].set_yscale('log')\n",
    "    axes[1, 0].legend()\n",
    "    \n",
    "    # Final R² score\n",
    "    full_r2 = history_phase1.history['r2_score_tf'] + history_phase2.history['r2_score_tf']\n",
    "    full_val_r2 = history_phase1.history['val_r2_score_tf'] + history_phase2.history['val_r2_score_tf']\n",
    "    \n",
    "    axes[1, 1].plot(epochs, full_r2, label='Training R²')\n",
    "    axes[1, 1].plot(epochs, full_val_r2, label='Validation R²')\n",
    "    axes[1, 1].axvline(x=Config.INITIAL_EPOCHS, color='red', linestyle='--', alpha=0.7, label='Fine-tuning Start')\n",
    "    axes[1, 1].set_title('Complete Training R²')\n",
    "    axes[1, 1].set_xlabel('Epoch')\n",
    "    axes[1, 1].set_ylabel('R²')\n",
    "    axes[1, 1].legend()\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "    \n",
    "    # Final best metrics\n",
    "    final_best_loss = min(full_val_loss)\n",
    "    final_best_mae = min(full_val_mae)\n",
    "    final_best_r2 = max(full_val_r2)\n",
    "    \n",
    "    logger.info(f\"Final Best Metrics:\")\n",
    "    logger.info(f\"  Validation Loss: {final_best_loss:.4f}\")\n",
    "    logger.info(f\"  Validation MAE: {final_best_mae:.4f}\")\n",
    "    logger.info(f\"  Validation R²: {final_best_r2:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === MODEL EVALUATION ===\n",
    "\n",
    "logger.info(\"=== MODEL EVALUATION ===\")\n",
    "\n",
    "# Evaluate on test set\n",
    "test_results = model.evaluate(test_dataset, steps=len(test_paths) // Config.BATCH_SIZE, verbose=1)\n",
    "test_loss, test_mae, test_rmse, test_r2 = test_results\n",
    "\n",
    "logger.info(f\"Test Results:\")\n",
    "logger.info(f\"  Test Loss: {test_loss:.4f}\")\n",
    "logger.info(f\"  Test MAE: {test_mae:.4f}\")\n",
    "logger.info(f\"  Test RMSE: {test_rmse:.4f}\")\n",
    "logger.info(f\"  Test R²: {test_r2:.4f}\")\n",
    "\n",
    "# Generate predictions for visualization\n",
    "test_predictions = model.predict(test_dataset, steps=len(test_paths) // Config.BATCH_SIZE)\n",
    "\n",
    "# Denormalize predictions and actual values\n",
    "test_volumes_actual = test_volumes[:len(test_predictions)] * volume_std + volume_mean\n",
    "test_predictions_denorm = test_predictions.flatten() * volume_std + volume_mean\n",
    "\n",
    "# Calculate actual metrics on denormalized data\n",
    "mae_actual = mean_absolute_error(test_volumes_actual, test_predictions_denorm)\n",
    "mse_actual = mean_squared_error(test_volumes_actual, test_predictions_denorm)\n",
    "rmse_actual = np.sqrt(mse_actual)\n",
    "r2_actual = r2_score(test_volumes_actual, test_predictions_denorm)\n",
    "\n",
    "logger.info(f\"Actual Volume Metrics:\")\n",
    "logger.info(f\"  MAE: {mae_actual:.2f} cm³\")\n",
    "logger.info(f\"  RMSE: {rmse_actual:.2f} cm³\")\n",
    "logger.info(f\"  R²: {r2_actual:.4f}\")\n",
    "\n",
    "# Visualization\n",
    "fig, axes = plt.subplots(2, 2, figsize=(15, 12))\n",
    "\n",
    "# Prediction vs Actual scatter plot\n",
    "axes[0, 0].scatter(test_volumes_actual, test_predictions_denorm, alpha=0.6)\n",
    "axes[0, 0].plot([test_volumes_actual.min(), test_volumes_actual.max()], \n",
    "                [test_volumes_actual.min(), test_volumes_actual.max()], 'r--', lw=2)\n",
    "axes[0, 0].set_xlabel('Actual Volume (cm³)')\n",
    "axes[0, 0].set_ylabel('Predicted Volume (cm³)')\n",
    "axes[0, 0].set_title(f'Predictions vs Actual (R² = {r2_actual:.3f})')\n",
    "axes[0, 0].grid(True, alpha=0.3)\n",
    "\n",
    "# Residuals plot\n",
    "residuals = test_predictions_denorm - test_volumes_actual\n",
    "axes[0, 1].scatter(test_volumes_actual, residuals, alpha=0.6)\n",
    "axes[0, 1].axhline(y=0, color='r', linestyle='--')\n",
    "axes[0, 1].set_xlabel('Actual Volume (cm³)')\n",
    "axes[0, 1].set_ylabel('Residuals (cm³)')\n",
    "axes[0, 1].set_title('Residuals Plot')\n",
    "axes[0, 1].grid(True, alpha=0.3)\n",
    "\n",
    "# Error distribution\n",
    "axes[1, 0].hist(residuals, bins=30, alpha=0.7, edgecolor='black')\n",
    "axes[1, 0].axvline(x=0, color='r', linestyle='--')\n",
    "axes[1, 0].set_xlabel('Residuals (cm³)')\n",
    "axes[1, 0].set_ylabel('Frequency')\n",
    "axes[1, 0].set_title('Error Distribution')\n",
    "axes[1, 0].grid(True, alpha=0.3)\n",
    "\n",
    "# Percentage error\n",
    "percentage_error = np.abs(residuals) / test_volumes_actual * 100\n",
    "axes[1, 1].hist(percentage_error, bins=30, alpha=0.7, edgecolor='black')\n",
    "axes[1, 1].set_xlabel('Absolute Percentage Error (%)')\n",
    "axes[1, 1].set_ylabel('Frequency')\n",
    "axes[1, 1].set_title(f'Percentage Error Distribution (Mean: {np.mean(percentage_error):.1f}%)')\n",
    "axes[1, 1].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()\n",
    "\n",
    "# Performance summary\n",
    "performance_summary = {\n",
    "    'Model': 'EfficientNetB0 + Custom Head',\n",
    "    'Training Samples': len(train_paths),\n",
    "    'Validation Samples': len(val_paths),\n",
    "    'Test Samples': len(test_paths),\n",
    "    'Test MAE (cm³)': f'{mae_actual:.2f}',\n",
    "    'Test RMSE (cm³)': f'{rmse_actual:.2f}',\n",
    "    'Test R²': f'{r2_actual:.4f}',\n",
    "    'Mean Absolute % Error': f'{np.mean(percentage_error):.1f}%',\n",
    "    'Training Time': 'Phase 1 + Phase 2',\n",
    "    'Hardware': 'RTX 3050 + Ryzen 7 6000'\n",
    "}\n",
    "\n",
    "print(\"\\n=== PERFORMANCE SUMMARY ===\")\n",
    "for key, value in performance_summary.items():\n",
    "    print(f\"{key:25}: {value}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# === SAVE FINAL MODEL ===\n",
    "\n",
    "class VolumePredictor(tf.keras.Model):\n",
    "    \"\"\"Wrapper class for production deployment\"\"\"\n",
    "    \n",
    "    def __init__(self, model, volume_mean, volume_std):\n",
    "        super().__init__()\n",
    "        self.model = model\n",
    "        self.volume_mean = volume_mean\n",
    "        self.volume_std = volume_std\n",
    "    \n",
    "    @tf.function\n",
    "    def call(self, inputs):\n",
    "        # Get normalized prediction\n",
    "        normalized_pred = self.model(inputs)\n",
    "        # Denormalize\n",
    "        actual_volume = normalized_pred * self.volume_std + self.volume_mean\n",
    "        return actual_volume\n",
    "    \n",
    "    def predict_volume(self, image_path):\n",
    "        \"\"\"Predict volume from image path\"\"\"\n",
    "        # Load and preprocess image\n",
    "        image = tf.io.read_file(image_path)\n",
    "        image = tf.image.decode_image(image, channels=3)\n",
    "        image = tf.image.resize(image, [Config.IMG_SIZE, Config.IMG_SIZE])\n",
    "        image = tf.cast(image, tf.float32)\n",
    "        image = tf.expand_dims(image, 0)  # Add batch dimension\n",
    "        \n",
    "        # Predict\n",
    "        volume = self.call(image)\n",
    "        return float(volume[0, 0])\n",
    "\n",
    "\n",
    "# Create production model\n",
    "production_model = VolumePredictor(model, volume_mean, volume_std)\n",
    "\n",
    "# Save the base model\n",
    "model.save(Config.MODEL_SAVE_PATH)\n",
    "logger.info(f\"✅ Model saved to: {Config.MODEL_SAVE_PATH}\")\n",
    "\n",
    "# Save model parameters for future use\n",
    "model_params = {\n",
    "    'volume_mean': float(volume_mean),\n",
    "    'volume_std': float(volume_std),\n",
    "    'img_size': Config.IMG_SIZE,\n",
    "    'model_architecture': 'EfficientNetB0',\n",
    "    'test_mae': float(mae_actual),\n",
    "    'test_rmse': float(rmse_actual),\n",
    "    'test_r2': float(r2_actual),\n",
    "    'training_samples': len(train_paths),\n",
    "    'validation_samples': len(val_paths),\n",
    "    'test_samples': len(test_paths)\n",
    "}\n",
    "\n",
    "# Save parameters\n",
    "params_path = Config.MODEL_SAVE_PATH.replace('.h5', '_params.npz')\n",
    "np.savez(params_path, **model_params)\n",
    "logger.info(f\"✅ Model parameters saved to: {params_path}\")\n",
    "\n",
    "# Test the production model\n",
    "if test_paths:\n",
    "    test_image_path = test_paths[0]\n",
    "    predicted_volume = production_model.predict_volume(test_image_path)\n",
    "    actual_volume = test_volumes_actual[0]\n",
    "    \n",
    "    logger.info(f\"\\nProduction Model Test:\")\n",
    "    logger.info(f\"  Image: {os.path.basename(test_image_path)}\")\n",
    "    logger.info(f\"  Predicted: {predicted_volume:.2f} cm³\")\n",
    "    logger.info(f\"  Actual: {actual_volume:.2f} cm³\")\n",
    "    logger.info(f\"  Error: {abs(predicted_volume - actual_volume):.2f} cm³\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*50)\n",
    "print(\"🎉 MODEL TRAINING COMPLETED SUCCESSFULLY!\")\n",
    "print(\"=\"*50)\n",
    "print(f\"📁 Model saved: {Config.MODEL_SAVE_PATH}\")\n",
    "print(f\"📊 Final Test R²: {r2_actual:.4f}\")\n",
    "print(f\"📏 Final Test MAE: {mae_actual:.2f} cm³\")\n",
    "print(f\"🎯 Mean Absolute % Error: {np.mean(percentage_error):.1f}%\")\n",
    "print(\"\\nThe model is ready for deployment! 🚀\")"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}



NameError: name 'null' is not defined